# Predicción de Cantidades Vendidas por Región y Categoría con Red Neuronal (MLP)

Este notebook realiza la predicción de cantidades vendidas por región y categoría utilizando una red neuronal (MLP), replicando la lógica de los modelos y vistas implementados en Snowflake. El flujo sigue los siguientes pasos:

- **Carga de datos** desde las vistas `VW_VENTAS_HISTORICO_M3` (histórico), `VW_VENTAS_FCST_FEATURES_1` (escenario 1) y `VW_VENTAS_FCST_FEATURES_2` (escenario 2) en Snowflake.
- **Preprocesamiento**: Manejo de valores nulos, codificación de variables categóricas, creación de características temporales y escalado de datos para los tres datasets.
- **Selección de características**: Uso de Recursive Feature Elimination (RFE) para identificar las características más importantes.
- **Entrenamiento del modelo**: Optimización de hiperparámetros mediante GridSearchCV sobre el histórico.
- **Predicciones**: Generación de predicciones para los dos escenarios de forecast, con intervalos de confianza del 95% por región y categoría.
- **Métricas**: Cálculo de RMSE, MAE, SMAPE y R² para entrenamiento, prueba y validación.
- **Almacenamiento**: Guardado de predicciones en la tabla `FORECAST_M3_MLP` y métricas en Snowflake.
- **Visualización**: Gráficas interactivas con Plotly mostrando el histórico y ambos escenarios de pronóstico por región y categoría.

**Objetivo**: Generar pronósticos de cantidades vendidas de producto en metros cúbicos por región y categoría, considerando variables temporales y otras características relevantes como precios, descuentos y márgenes de ganancia, bajo la misma óptica de los modelos Snowflake pero usando MLP.

## 1. Importar Librerías y Configuración

Importamos las librerías necesarias y configuramos el logging para trazabilidad.

In [3]:
import pandas as pd
import numpy as np
import snowflake.connector
import logging
from datetime import datetime
import os
from sklearn.model_selection import train_test_split, GridSearchCV, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.neural_network import MLPRegressor
import joblib
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from tqdm import tqdm
import matplotlib.pyplot as plt
import warnings

# Configurar advertencias y logging
warnings.filterwarnings("ignore")
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler()
    ]
)


c:\Users\joey_\AppData\Local\Programs\Python\Python310\lib\site-packages\snowflake\connector\options.py:104: UserWarning: You have an incompatible version of 'pyarrow' installed (19.0.1), please install a version that adheres to: 'pyarrow<19.0.0; extra == "pandas"'
  warn_incompatible_dep(


## 2. Carga de Datos desde Snowflake

Cargamos los datos históricos y los dos escenarios de forecast desde las vistas generadas en Snowflake:
- `VW_VENTAS_HISTORICO_M3` (histórico)
- `VW_VENTAS_FCST_FEATURES_1` (escenario 1: precio promedio y descuento promedio)
- `VW_VENTAS_FCST_FEATURES_2` (escenario 2: último precio con 20% incremento y descuento promedio)

Esto permite replicar el flujo de predicción de Snowflake en Python con MLP.

In [ ]:
def get_snowflake_connection():
    try:
        conn = snowflake.connector.connect(
        user='XXXXX',  
        password='XXXXX',  
        account='XXXXX', 
        warehouse='COMPUTE_WH', 
        database='BEBIDAS_PROJECT',
        schema='BEBIDAS_ANALYTICS'
        )
        logging.info("Conexión a Snowflake exitosa")
        return conn
    except Exception as e:
        logging.error(f"Error de conexión a Snowflake: {e}")
        raise
    
# Cargar los tres datasets desde Snowflake

def load_snowflake_views():
    conn = get_snowflake_connection()
    try:
        # Histórico
        df_hist = pd.read_sql("SELECT * FROM VW_VENTAS_HISTORICO_M3", conn)
        # Escenario 1
        df_fcst1 = pd.read_sql("SELECT * FROM VW_VENTAS_FCST_FEATURES_1", conn)
        # Escenario 2
        df_fcst2 = pd.read_sql("SELECT * FROM VW_VENTAS_FCST_FEATURES_2", conn)
        logging.info(f"Histórico: {len(df_hist)} registros, Escenario 1: {len(df_fcst1)}, Escenario 2: {len(df_fcst2)}")
        return df_hist, df_fcst1, df_fcst2
    finally:
        conn.close()

df_hist, df_fcst1, df_fcst2 = load_snowflake_views()
print(f"Histórico: {df_hist.shape}, Escenario 1: {df_fcst1.shape}, Escenario 2: {df_fcst2.shape}")
df_hist.head()

2025-06-28 15:29:43,279 - INFO - Snowflake Connector for Python Version: 3.15.0, Python Version: 3.10.11, Platform: Windows-10-10.0.19045-SP0
2025-06-28 15:29:43,281 - INFO - Connecting to GLOBAL Snowflake domain
2025-06-28 15:29:47,078 - INFO - Conexión a Snowflake exitosa
2025-06-28 15:29:58,228 - INFO - Histórico: 3984 registros, Escenario 1: 1680, Escenario 2: 1680


Histórico: (3984, 6), Escenario 1: (1680, 5), Escenario 2: (1680, 5)


,MES,REGION_CATEGORIA_PRODUCTO,M3_VENDIDOS,AVG_PRECIO_LISTA,AVG_DESC_PORCENTAJE,ORIGEN
0,2023-01-01,Antioquia-Bebida de Té-Té Negro 500mL x 6uds,1.5750,168000.0,3.630476,Histórico
1,2023-01-01,Antioquia-Gaseosa-Uva 600mL x 24uds,5.2704,50400.0,6.090164,Histórico
2,2023-01-01,Antioquia-Jugo-Jugo Naranja 2000mL x 24uds,21.7920,1800.0,4.522026,Histórico
3,2023-01-01,Antioquia-Agua-Agua Sin Gas 1L x 1uds,0.4850,2500.0,4.628866,Histórico
4,2023-01-01,Antioquia-Jugo-Jugo Lima 2000mL x 12uds,9.5760,96000.0,3.609023,Histórico


In [ ]:
df_hist = df_hist[['REGION_CATEGORIA_PRODUCTO']]